In [1]:
import os
import pathlib
import shutil
from PIL import Image
from pathlib import Path
from typing import Tuple, Dict, List
import pandas as pd
import numpy as np
import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
from timeit import default_timer as timer
from tqdm import tqdm

In [ ]:
# config

# specify main data path
PATH = "./data"

# specify the paths to the directory where the data is stored
TRAIN = "./data/train"
VALID = ".data/val"

# specify input height and width

HEIGHT = 128
WIDTH = 128

# set batch size and split

BATCH_SIZE = 32
VAL_SPLIT = 0.1

In [9]:
# Define the transforms
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# Load the dataset
dataset = datasets.ImageFolder(root='/Users/brandonling/Projects/colon-cancer-classification-using-cnn/data/', transform=transform)

# Check the size of the images
for img, _ in dataset:
    print(img.size())
    break  # Remove this line if you want to check all images

torch.Size([3, 128, 128])
